In [1]:
:load DescriptiveStats

In [33]:
import Data.List
import Data.Maybe
import Database.HDBC
import Database.HDBC.Sqlite3
import Graphics.EasyPlot
import DescriptiveStats

In [3]:
db <- connectSqlite3 "../data/movies.sqlite3"

In [4]:
quickQuery db "SELECT avg(rating) FROM data" []

[[SqlDouble 3.52986]]

In [5]:
quickQuery db "SELECT median(rating) FROM data" []

SqlError {seState = "", seNativeError = 1, seErrorMsg = "prepare 32: SELECT median(rating) FROM data: no such function: median"}

In [6]:
quickQuery db "SELECT mode(rating) FROM data" []

SqlError {seState = "", seNativeError = 1, seErrorMsg = "prepare 30: SELECT mode(rating) FROM data: no such function: mode"}

In [7]:
ratingsAction <- quickQuery db "SELECT data.rating FROM data, items WHERE data.itemid=items.movieid AND items.action=1" []

In [8]:
length ratingsAction

25589

In [9]:
readColumn = map fromSql

In [10]:
mean (readColumn ( (head . transpose) ratingsAction) :: [Double])

Just 3.480245417953027

In [11]:
median (readColumn ( (head . transpose) ratingsAction) :: [Double])

Just 4.0

In [12]:
ratingsDrama <- quickQuery db "SELECT data.rating FROM data, items WHERE data.itemid=items.movieid AND items.drama=1" []

In [13]:
mean (readColumn ( (head . transpose) ratingsDrama) :: [Double])

Just 3.6873793708484772

In [14]:
median (readColumn ( (head . transpose) ratingsDrama) :: [Double])

Just 4.0

In [15]:
ratingsScifi <- quickQuery db "SELECT data.rating FROM data, items WHERE data.itemid=items.movieid AND items.scifi=1" []

In [16]:
mean (readColumn ( (head . transpose) ratingsScifi) :: [Double])

Just 3.5607227022780834

In [17]:
median (readColumn ( (head . transpose) ratingsScifi) :: [Double])

Just 4.0

In [18]:
ageRatings <- quickQuery db "SELECT users.age, avg(data.rating) FROM data, users WHERE data.userid=users.userid GROUP BY users.age" []

In [19]:
age = readColumn (((!! 0) . transpose) ageRatings) :: [Double]

In [20]:
avgRating = readColumn (((!! 1) . transpose) ageRatings) :: [Double]

In [21]:
plot X11 $ zip age avgRating

True

In [22]:
plot X11 $ Data2D [Title "Age vs Avg. Rating"] [] (zip age avgRating)

True

In [34]:
kde :: [Double] -> Maybe ([Double], [Double])
kde [] = Nothing
kde xs = Just (domain, mykde)
  where
     mykde = map (/ sum shape) shape
     shape = foldl1 (zipWith (+)) (map (\x -> (map (normal x 1) domain)) xs)
     low = -5 + fst highlow
     high = 5 + snd highlow
     highlow = fromJust (range xs)
     domain = [low,(low+0.1)..high]
     normal mu sd x = exp (-((x-mu)^2 / (2*sd*sd))) / sqrt (2*sd*sd*pi)

In [35]:
quickQuery db  "SELECT data.itemid, items.title FROM data, items WHERE data.itemid=items.movieid GROUP BY data.itemid ORDER BY count(data.rating) DESC LIMIT 10" []

[[SqlInt64 50,SqlByteString "Star Wars (1977)"],[SqlInt64 258,SqlByteString "Contact (1997)"],[SqlInt64 100,SqlByteString "Fargo (1996)"],[SqlInt64 181,SqlByteString "Return of the Jedi (1983)"],[SqlInt64 294,SqlByteString "Liar Liar (1997)"],[SqlInt64 286,SqlByteString "English Patient, The (1996)"],[SqlInt64 288,SqlByteString "Scream (1996)"],[SqlInt64 1,SqlByteString "Toy Story (1995)"],[SqlInt64 300,SqlByteString "Air Force One (1997)"],[SqlInt64 121,SqlByteString "Independence Day (ID4) (1996)"]]

In [36]:
starwarsRaw <- quickQuery db "SELECT rating FROM data WHERE itemid=50" []

In [37]:
starwars = readColumn (((!! 0) . transpose) starwarsRaw) :: [Double]

In [38]:
length starwars

583

In [39]:
liarliarRaw <- quickQuery db "SELECT rating FROM data WHERE itemid=294" []

In [40]:
liarliar = readColumn (((!! 0) . transpose) liarliarRaw) :: [Double]

In [41]:
length liarliar

485

In [42]:
Just (domainSW, mykdeSW) = kde starwars

In [43]:
Just (domainLL, mykdeLL) = kde liarliar

In [45]:
plot X11 [Data2D [Style Lines] [] (zip domainSW mykdeSW), Data2D [Style Lines] [] (zip domainLL mykdeLL)]

True

In [46]:
domainSW

[-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.4999999999999996,-3.3999999999999995,-3.2999999999999994,-3.1999999999999993,-3.099999999999999,-2.999999999999999,-2.899999999999999,-2.799999999999999,-2.699999999999999,-2.5999999999999988,-2.4999999999999987,-2.3999999999999986,-2.2999999999999985,-2.1999999999999984,-2.0999999999999983,-1.9999999999999982,-1.8999999999999981,-1.799999999999998,-1.699999999999998,-1.5999999999999979,-1.4999999999999978,-1.3999999999999977,-1.2999999999999976,-1.1999999999999975,-1.0999999999999974,-0.9999999999999973,-0.8999999999999972,-0.7999999999999972,-0.6999999999999971,-0.599999999999997,-0.4999999999999969,-0.3999999999999968,-0.2999999999999967,-0.19999999999999662,-9.999999999999654e-2,3.552713678800501e-15,0.10000000000000364,0.20000000000000373,0.3000000000000038,0.4000000000000039,0.500000000000004,0.6000000000000041,0.7000000000000042,0.8000000000000043,0.9000000000000044,1.0000000000000044,1.1000000000000045,1.200000000000004

In [47]:
domainLL

[-4.0,-3.9,-3.8,-3.6999999999999997,-3.5999999999999996,-3.4999999999999996,-3.3999999999999995,-3.2999999999999994,-3.1999999999999993,-3.099999999999999,-2.999999999999999,-2.899999999999999,-2.799999999999999,-2.699999999999999,-2.5999999999999988,-2.4999999999999987,-2.3999999999999986,-2.2999999999999985,-2.1999999999999984,-2.0999999999999983,-1.9999999999999982,-1.8999999999999981,-1.799999999999998,-1.699999999999998,-1.5999999999999979,-1.4999999999999978,-1.3999999999999977,-1.2999999999999976,-1.1999999999999975,-1.0999999999999974,-0.9999999999999973,-0.8999999999999972,-0.7999999999999972,-0.6999999999999971,-0.599999999999997,-0.4999999999999969,-0.3999999999999968,-0.2999999999999967,-0.19999999999999662,-9.999999999999654e-2,3.552713678800501e-15,0.10000000000000364,0.20000000000000373,0.3000000000000038,0.4000000000000039,0.500000000000004,0.6000000000000041,0.7000000000000042,0.8000000000000043,0.9000000000000044,1.0000000000000044,1.1000000000000045,1.200000000000004

In [48]:
sum $ map (\(sw, ll) -> if ll <= sw then ll else sw) (zip mykdeSW mykdeLL)

0.6533607763536907

In [49]:
sum $ map (\(sw, ll) -> if ll <= sw then ll else sw) (zip mykdeSW mykdeSW)

1.0000000000000002